<a href="https://colab.research.google.com/github/Vicyn666/IncomePredictionRBGroup1/blob/main/Group1_Income_pred_Task1_21feb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Решение задачи определения уровня дохода


## <font color='green'>1. Загрузка данных и их первичный анализ</font>

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

In [2]:
!mkdir -p ~/.postgresql && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
    --output-document ~/.postgresql/root.crt && \
chmod 0600 ~/.postgresql/root.crt

--2023-02-20 17:44:51--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3.5K) [application/x-x509-ca-cert]
Saving to: ‘/root/.postgresql/root.crt’

/root/.postgresql/r 100%[===================>]   3.50K  --.-KB/s    in 0s      

2023-02-20 17:44:52 (624 MB/s) - ‘/root/.postgresql/root.crt’ saved [3579/3579]



In [3]:
!sudo apt update && sudo apt install --yes python3 python3-pip && \
!pip3 install psycopg2-binary

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/p

In [4]:
import psycopg2

conn = psycopg2.connect("""
    host=rc1b-70q90cunwkzx49mj.mdb.yandexcloud.net
    port=6432
    sslmode=verify-full
    dbname=db_clients
    user=mlhs_student
    password=mlhs_student
    target_session_attrs=read-write
""")

In [5]:
import pandas as pd

query = "SELECT * from adult_income"
data = pd.read_sql_query(query, conn)

conn.close()

In [6]:
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K


In [7]:
data.shape

(9615, 15)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9615 entries, 0 to 9614
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              9615 non-null   int64  
 1   workclass        9615 non-null   object 
 2   fnlwgt           9615 non-null   int64  
 3   education        9615 non-null   object 
 4   educational-num  9615 non-null   int64  
 5   marital-status   9615 non-null   object 
 6   occupation       9615 non-null   object 
 7   relationship     9615 non-null   object 
 8   race             9614 non-null   object 
 9   gender           9614 non-null   object 
 10  capital-gain     9614 non-null   float64
 11  capital-loss     9614 non-null   float64
 12  hours-per-week   9614 non-null   float64
 13  native-country   9614 non-null   object 
 14  income           9614 non-null   object 
dtypes: float64(3), int64(3), object(9)
memory usage: 1.1+ MB


Удаляем строку с пустым значением поля income

In [11]:
data=data[data.income.notnull()]

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9614 entries, 0 to 9613
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              9614 non-null   int64  
 1   workclass        9614 non-null   object 
 2   fnlwgt           9614 non-null   int64  
 3   education        9614 non-null   object 
 4   educational-num  9614 non-null   int64  
 5   marital-status   9614 non-null   object 
 6   occupation       9614 non-null   object 
 7   relationship     9614 non-null   object 
 8   race             9614 non-null   object 
 9   gender           9614 non-null   object 
 10  capital-gain     9614 non-null   float64
 11  capital-loss     9614 non-null   float64
 12  hours-per-week   9614 non-null   float64
 13  native-country   9614 non-null   object 
 14  income           9614 non-null   object 
dtypes: float64(3), int64(3), object(9)
memory usage: 1.2+ MB


Проверяем, что пустых значений больше нет

In [13]:
data.isna().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               0
gender             0
capital-gain       0
capital-loss       0
hours-per-week     0
native-country     0
income             0
dtype: int64

Заменим категориальные признаки в поле income на числовые: 



In [14]:
data['income'] = data['income'].replace('<=50K', 0)
data['income'] = data['income'].replace('>50K', 1)

Удаляем категориальные признаки

In [15]:
data = data.drop(['workclass','education','marital-status','occupation','relationship','race','gender','native-country'],axis=1)

In [16]:
data.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income
0,25,226802,7,0.0,0.0,40.0,0
1,38,89814,9,0.0,0.0,50.0,0
2,28,336951,12,0.0,0.0,40.0,1
3,44,160323,10,7688.0,0.0,40.0,1
4,18,103497,10,0.0,0.0,30.0,0


Проверяем количество каждого из вариантов дохода

In [17]:
data['income'].value_counts()

0    7356
1    2258
Name: income, dtype: int64

In [18]:
data.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income
count,9614.000000,9.614000e+03,9614.000000,9614.000000,9614.000000,9614.000000,9614.000000
mean,38.731329,1.882948e+05,10.080092,1121.998232,87.926669,40.522987,0.234866
std,13.842687,1.051352e+05,2.562733,7890.578253,404.085647,12.416221,0.423937
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,1.154652e+05,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,1.778450e+05,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,2.374175e+05,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,3770.000000,99.000000,1.000000


Строим матрицу корреляции

In [19]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income
age,1.000000,-0.081938,0.006905,0.082360,0.052424,0.081728,0.224250
fnlwgt,-0.081938,1.000000,-0.024398,-0.005989,-0.000409,-0.004686,-0.003089
educational-num,0.006905,-0.024398,1.000000,0.132639,0.083546,0.136237,0.320775
capital-gain,0.082360,-0.005989,0.132639,1.000000,-0.030944,0.098911,0.221998
capital-loss,0.052424,-0.000409,0.083546,-0.030944,1.000000,0.056343,0.139776
hours-per-week,0.081728,-0.004686,0.136237,0.098911,0.056343,1.000000,0.228856
income,0.224250,-0.003089,0.320775,0.221998,0.139776,0.228856,1.000000


Корреляция с признаком fnlwgt крайне слабая, удалим его

In [20]:
data = data.drop(['fnlwgt'],axis=1)

In [21]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,age,educational-num,capital-gain,capital-loss,hours-per-week,income
age,1.000000,0.006905,0.082360,0.052424,0.081728,0.224250
educational-num,0.006905,1.000000,0.132639,0.083546,0.136237,0.320775
capital-gain,0.082360,0.132639,1.000000,-0.030944,0.098911,0.221998
capital-loss,0.052424,0.083546,-0.030944,1.000000,0.056343,0.139776
hours-per-week,0.081728,0.136237,0.098911,0.056343,1.000000,0.228856
income,0.224250,0.320775,0.221998,0.139776,0.228856,1.000000


## <font color='green'>2.1. Обучение модели</font>



Сформируем матрицу X объект-признак и вектор y целевой переменной.

In [22]:
X = data.drop('income', axis=1) # матрица объект-признак
y = data['income'] # целевая переменная

In [23]:
X_full = X.copy()
#X = X.drop(['state','areacode','voicemailplan','internationalplan'], axis=1)

Мы разбиваем данные на train и test, чтобы:
* на тренировочных данных обучить модель
* на тестовых применить обученную модель и затем проверить качество модели

In [24]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=42)

Для линейных моделей очень важно масштабирование признаков. Поэтому отмасштабируем признаки при помощи StandardScaler, обучив метод на тренировочных данных и применив его и к трейну, и к тесту.

In [25]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(Xtrain)

Xtrain = pd.DataFrame(ss.transform(Xtrain), columns=X.columns)
Xtest = pd.DataFrame(ss.transform(Xtest), columns=X.columns)

Xtrain.head()

,age,educational-num,capital-gain,capital-loss,hours-per-week
0,-1.436662,-0.032251,-0.145108,-0.221089,-2.160013
1,-1.291984,-0.032251,-0.145108,-0.221089,-0.043911
2,-1.002627,-0.419154,-0.145108,-0.221089,0.769974
3,-0.568591,1.128459,-0.145108,-0.221089,-0.450854
4,-0.423912,1.128459,-0.145108,-0.221089,1.176916


Обучим логистическую регрессию на train и сделаем предсказание на новых для нее данных (test).

In [26]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression() # объявление модели
model.fit(Xtrain, ytrain) # обучение модели

LogisticRegression()

In [27]:
pred_test = model.predict(Xtest) # предсказание классов

In [28]:
accuracy_score(ytest, pred_test)

0.8157237936772047

In [29]:
y.value_counts()

0    7356
1    2258
Name: income, dtype: int64

Баланса классов нет: большинство имеют доход менее <=50K (77%)

Посмотрим, что предсказывает модель.

In [30]:
pred_test[:50]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

## <font color='green'>2.2. Валидация модели</font>

Для оценки качества модели используются различные метрики качества. Давайте посмотрим, какие метрики полезно считать в задаче классификации, кроме accuracy.

<font color='blue'>Для оценки качества моделей бинарной классификации полезно смотреть на **матрицу ошибок** (confusion matrix):</font>

In [31]:
from sklearn.metrics import confusion_matrix

confusion_matrix(ytest, pred_test)

array([[1768,   74],
       [ 369,  193]])

In [32]:
ytest.value_counts()

0    1842
1     562
Name: income, dtype: int64

In [33]:
from sklearn.metrics import recall_score

recall_score(ytest, pred_test)

0.34341637010676157

In [34]:
from sklearn.metrics import precision_score

precision_score(ytest, pred_test)

0.7228464419475655